In [1]:
# h_np
import numpy as np
import pickle
from datetime import datetime

import sys
import os
sys.path.append('../')

# local imports
import envs

import torch
from np_hybrid_stochastic import PathIntegral
from model import ModelOptimizer, Model, SARSAReplayBuffer
# from model import MDNModelOptimizer, MDNModel

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

current_dir=/home/allison/.local/lib/python3.6/site-packages/pybullet_envs/bullet


In [30]:
# planner
import torch
from torch.distributions import Normal

class PathIntegral(object):

    def __init__(self, model, samples=10, t_H=10, lam=0.1):

        self.model          = model
        self.num_actions    = model.num_actions
        self.t_H            = t_H
        self.lam            = lam
        self.samples        = samples

        self.a = torch.zeros(t_H, self.num_actions)

    def reset(self):
        self.a.zero_()

    def __call__(self, state):

        with torch.no_grad(): # temporarily set all the requires_grad flag to false
            self.a[:-1] = self.a[1:].clone() # creates a copy of tensor that imitates the original tensor's requires_grad field
            self.a[-1].zero_()


            s0 = torch.FloatTensor(state).unsqueeze(0)
            s = s0.repeat(self.samples, 1)
            # mu, log_std = self.policy(s)
            sk = []
            da = []
            log_prob = []
            for t in range(self.t_H):
                pi = Normal(mu, log_std.exp())
                v = pi.sample() # current action + random noise
                log_prob.append(pi.log_prob(self.a[t].expand_as(v)).sum(1))
                da.append(v - self.a[t].expand_as(v))
                s, rew = self.model.step(s, v)
                # mu, log_std = self.policy(s)
                sk.append(-rew.squeeze())

            sk = torch.stack(sk)
            sk = torch.cumsum(sk.flip(0), 0).flip(0)

            sk = sk - torch.min(sk, dim=1, keepdim=True)[0]

            log_prob = torch.stack(log_prob)
            log_prob -= torch.max(log_prob, dim=1, keepdim=True)[0]

            w = torch.exp(-sk.div(self.lam) + log_prob) + 1e-5
            w.div_(torch.sum(w, dim=1, keepdim=True))
            for t in range(self.t_H):
                self.a[t] = self.a[t] + torch.mv(da[t].T, w[t])

            return self.a[0].clone().numpy()


In [35]:
class struct(object):
    pass

# arguments
args = struct()
args.env = 'PendulumEnv'
args.max_steps = 200 
args.max_frames = 6000
args.frame_skip = 4
args.model_lr = 3e-3
# args.value_lr = 3e-4
# args.soft_q_lr = 3e-4

args.horizon = 5
args.model_iter = 2
args.trajectory_samples = 20
args.done_util = True
args.render = True

In [36]:
# main
env_name = args.env
try:
    env = envs.env_list[env_name](render=args.render)
except TypeError as err:
    print('no argument render, assuming env.render will just work')
    env = envs.env_list[env_name]()
env.reset()
print(env.action_space.low, env.action_space.high)
assert np.any(np.abs(env.action_space.low) <= 1.) and  np.any(np.abs(env.action_space.high) <= 1.), 'Action space not normalizd'


now = datetime.now()
date_str = now.strftime("%Y-%m-%d_%H-%M-%S/")

path = './data/' + env_name +  '/' + 'h_np/' + date_str
if os.path.exists(path) is False:
    os.makedirs(path)

action_dim = env.action_space.shape[0]
state_dim  = env.observation_space.shape[0]
hidden_dim = 128

# policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim)

model = Model(state_dim, action_dim, def_layers=[200])
# model = MDNModel(state_dim, action_dim, def_layers=[200, 200])


replay_buffer_size = 1000000
# replay_buffer = ReplayBuffer(replay_buffer_size)

model_replay_buffer = SARSAReplayBuffer(replay_buffer_size)
model_optim = ModelOptimizer(model, model_replay_buffer, lr=args.model_lr)

# model_optim = MDNModelOptimizer(model, replay_buffer, lr=args.model_lr)

planner = PathIntegral(model, samples=args.trajectory_samples, t_H=args.horizon, lam=0.1)

max_frames  = args.max_frames
max_steps   = args.max_steps
frame_skip = args.frame_skip

frame_idx   = 0
rewards     = []
batch_size  = 128

# env.camera_adjust()
ep_num = 0

no argument render, assuming env.render will just work
[-1.] [1.]


In [37]:
# while loop 
for k in range(200):
    state = env.reset()
    planner.reset()

    action = planner(state)

    episode_reward = 0
    for step in range(max_steps):
        # action = policy_net.get_action(state)
        for _ in range(frame_skip):
            next_state, reward, done, _ = env.step(action.copy())


        next_action = planner(next_state)

        replay_buffer.push(state, action, reward, next_state, done)
        model_replay_buffer.push(state, action, reward, next_state, next_action, done)

        if len(replay_buffer) > batch_size:
            # sac.soft_q_update(batch_size)
            model_optim.update_model(batch_size, mini_iter=args.model_iter)

        state = next_state
        action = next_action
        episode_reward += reward
        frame_idx += 1

        if args.render:
            env.render("human")


        if frame_idx % int(max_frames/10) == 0:
            print(
                'frame : {}/{}, \t last rew : {}, \t rew loss : {}'.format(
                    frame_idx, max_frames, rewards[-1][1], model_optim.log['rew_loss'][-1]
                )
            )

            pickle.dump(rewards, open(path + 'reward_data' + '.pkl', 'wb'))
            torch.save(policy_net.state_dict(), path + 'policy_' + str(frame_idx) + '.pt')

        if args.done_util:
            if done:
                break

RuntimeError: size mismatch, m1: [1 x 6], m2: [4 x 200] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:197

In [ ]:
ctrl.u

In [ ]:
x = torch.randn(5,2, requires_grad = True)

In [ ]:
x

In [ ]:
x[:-1] = x[1:].clone()

In [ ]:
x[-1].zero_()

In [ ]:
x